Для решения задач нам понадобится выделить из признака title год выпуска фильма. Для этого напишем функцию get_year_release(arg).

In [1]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

import pandas as pd
joined = pd.read_csv("data/ratings_movies.csv")    

Данная функция основана на регулярных выражениях и использует модуль re, с которым мы пока не работали.

→ Модуль re предназначен для поиска шаблонов в тексте и встроен в язык, поэтому не нуждается в установке.

Из модуля re нам понадобится только функция findall(), которая позволяет найти в строке все слова, удовлетворяющие шаблону. Мы находим в строке  
с названием фильма шаблон "(DDDD)" — четыре цифры, обёрнутых в скобки, что соответствует году выпуска фильма. Если такого шаблона не было найдено  
(год выпуска не указан), функция возвращает None (в таблице это будет помечено как пропуск).

Совет: для выполнения заданий используйте изученные в этом модуле методы.

In [2]:
# Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
# У скольких фильмов не указан год их выпуска?
joined['year_release'] = joined['title'].apply(get_year_release)
joined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


In [3]:
# Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
# В качестве ответа запишите название этого фильма без указания года его выпуска.
mask = joined['year_release'] == 1999
joined[mask].groupby('title')['rating'].mean().sort_values()

title
Bloodsport: The Dark Kumite (1999)                  0.5
Trippin' (1999)                                     1.0
Chill Factor (1999)                                 1.0
From Dusk Till Dawn 2: Texas Blood Money (1999)     1.0
Simon Sez (1999)                                    1.0
                                                   ... 
Mickey's Once Upon a Christmas (1999)               5.0
On the Ropes (1999)                                 5.0
Trailer Park Boys (1999)                            5.0
Five Senses, The (1999)                             5.0
George Carlin: You Are All Diseased (1999)          5.0
Name: rating, Length: 261, dtype: float64

In [4]:
# Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
# Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).
mask = joined['year_release'] == 2010
joined[mask].groupby('genres')['rating'].mean().sort_values()

genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Comedy|Musical                       5.000000
Name: rating, Length: 119, dtype: float64

In [5]:
# Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов?  
# В качестве ответа запишите идентификатор этого пользователя.
joined.groupby('userId')['genres'].nunique().sort_values(ascending=False)

userId
599    524
414    482
448    403
380    399
474    395
      ... 
494     15
578     15
214     13
85      13
245     13
Name: genres, Length: 610, dtype: int64

In [6]:
# Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
# В качестве ответа укажите идентификатор этого пользователя.
# Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку),
# можно воспользоваться методом agg() на сгруппированных данных.

joined.groupby('userId')['rating'].agg(
    ['count', 'mean']
).sort_values(['count', 'mean'], ascending=[True, False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [7]:
# Найдите сочетание жанров (genres) за 2018 год, которое имеет наибольший средний рейтинг (среднее по столбцу rating),
# и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.

mask = joined['year_release'] == 2018
grouped = joined[mask].groupby('genres')['rating'].agg(
    ['mean', 'count']
)
grouped[grouped['count']>10].sort_values(
    by='mean',
    ascending=False
)

,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


In [8]:
# Добавьте в таблицу новый признак year_rating — год выставления оценки. Создайте сводную таблицу, которая иллюстрирует зависимость
# среднего рейтинга фильма от года выставления оценки и жанра. Выберите верные варианты ответа, исходя из построенной таблицы:

joined['date'] = pd.to_datetime(joined['date'])
joined['year_rating'] = joined['date'].dt.year
pivot = joined.pivot_table(
    index='year_rating',
    columns='genres',
    values='rating',
    aggfunc='mean'
)
display(pivot)

genres,(no genres listed),Action,Action|Adventure,Action|Adventure|Animation,Action|Adventure|Animation|Children,Action|Adventure|Animation|Children|Comedy,Action|Adventure|Animation|Children|Comedy|Fantasy,Action|Adventure|Animation|Children|Comedy|IMAX,Action|Adventure|Animation|Children|Comedy|Romance,Action|Adventure|Animation|Children|Comedy|Sci-Fi,...,Romance|Thriller,Romance|War,Romance|Western,Sci-Fi,Sci-Fi|IMAX,Sci-Fi|Thriller,Sci-Fi|Thriller|IMAX,Thriller,War,Western
year_rating,,,,,,,,,,,,,,,,,,,,,
1996,NaN,2.730769,3.454545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.666667,NaN,3.838095,NaN,3.117647
1997,NaN,3.538462,4.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.400000,NaN,3.923077,NaN,3.000000
1998,NaN,NaN,4.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.800000,NaN,NaN
1999,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.000,NaN,NaN,NaN,NaN,4.000000,NaN,3.700000,4.5,4.000000
2000,NaN,2.588235,3.738462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.000,NaN,3.0,3.416667,NaN,2.142857,NaN,3.087912,3.0,4.058824
2001,NaN,3.000000,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,2.500000,NaN,2.500000,NaN,3.477273,3.0,3.111111
2002,NaN,2.750000,4.304348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.000,NaN,NaN,3.750000,NaN,3.600000,NaN,3.583333,3.5,3.000000
2003,NaN,3.833333,3.277778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.375,2.5,NaN,2.333333,NaN,3.142857,NaN,3.250000,3.0,4.000000
2004,NaN,2.700000,4.136364,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,...,3.000,3.0,3.5,2.125000,NaN,NaN,NaN,3.464286,3.0,3.800000


Дано два файла:

* orders.csv, содержащий данные о заказах;
* products.csv, содержащий данные о товарах.

Прочитайте данные файлы, посмотрите на содержимое таблиц и проанализируйте его.

Подумайте, как связаны данные таблицы (какими ключевыми столбцами).

Объедините заданные таблицы в таблицу orders_products, чтобы в результирующей таблице оказалась информация обо всех заказах,  
но не оказалось информации о продуктах, на которых заказов ещё не поступало. 

Ответьте на несколько вопросов по таблице orders_products ↓

In [10]:
import pandas as pd

orders = pd.read_csv("data/orders.csv")
orders_df = orders.copy()

product = pd.read_csv("data/products.csv", sep=';')
product_df = product.copy()

display(product_df.columns)
display(orders_df.columns)

Index(['Product_ID', 'Name', 'Price', 'CURRENCY'], dtype='object')

Index(['Дата создания;Order ID;ID Покупателя;Статус;Оплачен;Отменен;Отгружен;ID товара;Количество'], dtype='object')

In [ ]:
# Какой идентификатор (Order ID) имеет заказ, для которого не оказалось информации о товаре?
orders_products = orders_df.merge(
    product_df, 
    left_on='ID товара',
    right_on='Product_ID',
    how='left')
#orders_products[orders_products['Name'].isna()]

KeyError: 'ID товара'